# Introductory Econometrics: A modern approach
---

# Chapter 12: Serial Correlation and Heteroskedasticity in Time Series Regressions

> Book: 

    - WOOLDRIDGE, J. M.. Introdução à econometria: uma abordagem moderna. 3a ed. São Paulo: Pioneira Thomson Learning, 2006.

    - WOOLDRIDGE, Jeffrey M. Introductory econometrics: A modern approach. Cengage learning, 2015.

> LINK to codes correction:

http://www.upfie.net/downloads12.html

# Bibliotecas
---

In [1]:
import wooldridge as woo
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import patsy as pt

from statsmodels.stats.diagnostic import acorr_breusch_godfrey

# Propriedades dos estimadores de MQO com correlação serial 
---

1. **Viés e consistência:**

- Assim como no caso da heteroscedasticidade, não teremos estimadores inconsistentes caso haja correlação serial: enquanto as variáveis forem de fato exógenas, os estimadores serão **consistentes;**

- Contudo, pode haver viés em alguns casos: caso haja dependência fraca entre *x* e *u.*

2. **Eficiência e inferência:**

- Estimadores de MQO não são mais BLUE com correlação serial.;

- Os erros padrões dos estimadores e as estatísticas de teste não são mais válidas, nem assintóticamente.

- O estimador da variância estará sub ou sobre estimado, inviabilizando os testes estatísticos.

# Coeficiente de determinação ($R^2$)
---

- Se a série for estacionária ou com dependência fraca, o coeficiente de determinação ainda é válido, dado que a variância do erro e da variável dependente não mudam muito ao longo do tempo.

- Sem essas condições, ou com sua persistência mesmo com a série já diferenciada, a métrica perde seu sentido.

# Correlação serial com variáveis defasadas
---

- Mesmo com correlação serial, é possível obter estimadores consistentes, mas, ainda terão as estatísticas de teste inválidas.

- Os estimadores só se tornam **inconsistentes quando o erro também segue um processo autorregressivo.**

## Obs 1:

- Muitas vezes, a presença de correlação serial nada mais indica do que um modelo mal especificado, ou seja, acrescentar mais uma variável ou mais uma defasagem podem corrigir o problema, dependendo do caso.

# Testando para a correlação serial
---

## 1) Teste t para regressores estritamente exógenos

- $H_0$: não há correlação serial ($\rho_1 = 0$);

- $H_1$: há correlação serial($\rho_1 \neq 0$).

Devemos assumir que o valor esperado do erro, dado o histórico de variáveis independentes, é zero e que a variância é constante. 

Podemos observar e testar essa hipótese regredindo *u* em t em *u* em t-1 mais + um novo termo de erro *e*. Para esse teste, **usamos e estatística t**, já conhecida.

$$
u_t = \rho u_{t-1} + e_t
$$

Como não podemos observar o u, usaremos u chapéu, os resíduos da regressão com os dados da nossa amostra.

Em linhas gerais, tratamos a correlação serial como um **problema relevante** quando a Hipótese Nula é rejeitada em um nível de 5% (p-valor < 0.05, baixíssima probabilidade de errarmos ao rejeitar a hipótese nula).

**VALE LEMBRAR:** Importância estatística X Importância prática

- Mesmo se a hipótese nula for rejeitada e houver correlação serial, se seu valor estimado em for muito baixo, temos uma importância prática pouco relevante. Ou seja, usar MQO para estimar mesmo com correlação serial não nos traz grandes problemas práticos. 

## Exemplo 12.1: Testing for AR(1) Serial Correlation in the Phillips Curve

### Phillips Curve: expectations augmented

In [2]:
phillips = woo.dataWoo('phillips')
T = len(phillips)

# define yearly time series beginning in 1948:
date_range = pd.date_range(start='1948', periods=T, freq='Y')
phillips.index = date_range.year

# estimation of expectations-augmented Phillips curve:
yt96 = (phillips['year'] <= 1996)
phillips['inf_diff1'] = phillips['inf'].diff()
reg_ea = smf.ols(formula='inf_diff1 ~ unem', data=phillips, subset=yt96)
results_ea = reg_ea.fit()

phillips['resid_ea'] = results_ea.resid
phillips['resid_ea_lag1'] = phillips['resid_ea'].shift(1)
reg = smf.ols(formula='resid_ea ~ resid_ea_lag1', data=phillips, subset=yt96)
results = reg.fit()

# print regression table:
table = pd.DataFrame({'p': round(results.params, 4),
                      'se': round(results.bse, 4),
                      't': round(results.tvalues, 4),
                      'pval': round(results.pvalues, 4)})
print(f'table: \n{table}\n')

table: 
                    p      se       t    pval
Intercept      0.1942  0.3004  0.6464  0.5213
resid_ea_lag1 -0.0356  0.1239 -0.2873  0.7752



### Phillips Curve: static

In [3]:
phillips = woo.dataWoo('phillips')
T = len(phillips)

# define yearly time series beginning in 1948:
date_range = pd.date_range(start='1948', periods=T, freq='Y')
phillips.index = date_range.year

# estimation of static Phillips curve:
yt96 = (phillips['year'] <= 1996)
reg_s = smf.ols(formula='Q("inf") ~ unem', data=phillips, subset=yt96)
results_s = reg_s.fit()

# residuals and AR(1) test:
phillips['resid_s'] = results_s.resid
phillips['resid_s_lag1'] = phillips['resid_s'].shift(1)
reg = smf.ols(formula='resid_s ~ resid_s_lag1', data=phillips, subset=yt96)
results = reg.fit()

# print regression table:
table = pd.DataFrame({'p': round(results.params, 4),
                      'se': round(results.bse, 4),
                      't': round(results.tvalues, 4),
                      'pval': round(results.pvalues, 4)})
print(f'table: \n{table}\n')

table: 
                   p      se       t    pval
Intercept    -0.1134  0.3594 -0.3155  0.7538
resid_s_lag1  0.5730  0.1161  4.9337  0.0000



**OBS:** Apesar dos exemplos com a curva de Phillips e com o modelo AR(1), esse teste serve para detectar a correlação em qualquer especificação de regressão em séries temporais, justamente pelo seu formato. 

$$
u_t = \rho u_{t-1} + e_t
$$

Mas não necessariamente irá detectar em todas as situações: pode ser que o erro não seja correlacionado com sua defasagem imediatamente anterior (erros adjacentes), mas que esteja correlacionado com defasagens mais antigas, por conta de sazonalidade, por exemplo. 

**OBS 2:** Em geral, **assumimos que há homoscedasticidade** no termo do erro para essa regressão. Mas caso essa hipótese não se encaixe para o caso, podemos simplesmente utilizar a **estatística t e os erros padrões robustos**.

## 2) Teste de Durbin-Watson (DW) sob as hipóteses clássicas

- Teste também baseado nos **resíduos da regressão estimada com MQO.**

- Conceitualmente muito similar ao teste t com regressores estritamente exógenos.

- Esse teste demanda a “validade” (ou algo bem próximo) de todas as hipóteses clássicas para modelos de regressão linear em séries temporais, inclusive a normalidade dos erros.

- Dessa forma, depende do: **tamanho da amostra, quantidade de regressores e se a regressão contém o intercepto.**

- O conjunto de hipóteses é similar ao primeiro teste acima.

- Limites superiores e inferiores para a estatística de teste DW e com possibilidade de inconclusividade no teste.

- Apesar de possuir uma vantagem de computar a distribuição amostral da estatística DW é a única vantagem em relação ao método anterior. Mas suas dificuldades práticas podem deixar o primeiro teste como uma opção mais atrativa.

## 3) Teste para regressores sem exogeneidade estrita

- Sem exogeneidae estrita, nem o teste t e nem o Durbin-Watson serão válidos, mesmo em grandes amostras.

- Estatística alternativa de Durbin:
    - Obter os resídos da regressão de y nos x;

    - Regressão dos resíduos nos x e na primeira defasagem de u e obtenção do p e sua estatística de teste;

    - Usar t para testar a hipótese como ocorre no teste t relatado acima

- Incluir as variáveis explicativas nessa regressão explicita a relação direta que o resíduo e sua primeira defasagem podem ter com as variáveis exógenas, controlando para a endogeneidade das variáveis exógenas.

- Vale lembrar que essa estatística pode ser válida sob heteroscedasticidade utilizando sua versão robusta.

## Testando para correlação serial de ordem superior

- Os testes destacados acima podem facilmente ser estendidos para correlações seriais de ordem superior, ou seja, para mais defasagens no termo de erro

- Tomamos os mesmo procedimentos, mas agora, como estamos testando para a correlação serial em mais defasagens, podemos usar também o teste F como referencial, além do teste t.

- Há também uma versão do teste robusta a heteroscedasticidade

- Outra alternativa de teste seria utilizar a estatística do Multiplicador de Lagrange, conhecida também por **Breusch-Godfrey test**.

    - Esse teste também pode ser feito com robustes à heteroscedasticidade


- Para séries sem ajuste sazonal, sejam mensais ou trimestrais, geralmente podemos usar essa mesma estrutura de teste para investigar as defasagens no período em frequência t que se espera a sazonalidade.

# Corrigindo a correlação serial com regressores exógenos 
---

## Obtendo o melhor estimador linear não viesado

- Podemos usar os estimadores de mínimos quadrados generalizados (MQG)

- Os estimadores de MQG, ou seja, de MQO nos dados tranformados, são diferentes dos inicialmente estimados por MQO

- Estimadores de MQG se tornam BLUE e como os erros estão transformados, a equação não possui correlação serial e é homoscedastica;

- Os testes t e F são válidos

## Estimação factível de MQG

1. Estimação via MQO e obtenção dos resíduos

2. Regressão dos resíduos nas variáveis exógenas e em uma defasagem 

3. Aplicar a MQO para a equação com os dados transformados

4. Com esse processo, os testes t e F são assintóticamente válidos

- Assim, temos a estimação factível por MQG, o MQGF: mínimos quadrados generalizados factível

- Os estimadores de MQGF não são não-viesados, mas são mais eficientes assintóticamente do que os estimadores de MQO.

- Contudo, isso ocorre apenas se assumirmos que a série possui **dependência fraca**.

- Diferentes formas de se obter estimadores MQGF:

    - Cochrane-Orcutt (CO) estimation: omite a primeira observação

    - Prais-Winsten (PW) estimation: usa a primeira observação.

- Ambas as formas podem ser usadas de interativamente e repetimos as estimações até que os resultados sejam relativamente parecidos

## Example 12.4: Prais-Winsten Estimation in the Event Study

In [4]:
# Carregando os dados e criando a variável para o número de observações da série 

barium = woo.data('barium')
T = len(barium) 
barium

,chnimp,bchlimp,befile6,affile6,afdec6,befile12,affile12,afdec12,chempi,gas,...,apr,may,jun,jul,aug,sep,oct,nov,dec,percchn
0,220.462006,9578.375977,0,0,0,0,0,0,100.099998,7.830000e+09,...,0,0,0,0,0,0,0,0,0,2.301664
1,94.797997,11219.480469,0,0,0,0,0,0,100.900002,8.820000e+09,...,0,0,0,0,0,0,0,0,0,0.844941
2,219.357498,9719.900391,0,0,0,0,0,0,101.099998,8.450000e+09,...,1,0,0,0,0,0,0,0,0,2.256788
3,317.421509,12920.950195,0,0,0,0,0,0,102.500000,9.240001e+09,...,0,1,0,0,0,0,0,0,0,2.456642
4,114.639000,9790.446289,0,0,0,0,0,0,104.099998,9.150000e+09,...,0,0,1,0,0,0,0,0,0,1.170927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,589.421997,31686.859375,0,0,0,0,0,0,154.800003,9.420000e+09,...,0,0,0,0,1,0,0,0,0,1.860147
127,529.733521,16370.339844,0,0,0,0,0,0,155.300003,8.790000e+09,...,0,0,0,0,0,1,0,0,0,3.235935
128,968.037476,15653.959961,0,0,0,0,0,0,156.699997,8.990000e+09,...,0,0,0,0,0,0,1,0,0,6.183978
129,794.059998,45716.609375,0,0,0,0,0,0,157.500000,8.920001e+09,...,0,0,0,0,0,0,0,1,0,1.736918


In [5]:
# Série temporal mensal, selecionando o começo a partir de fevereiro de 1978

barium.index = pd.date_range(start = '1978-02', periods = T, freq = 'M')

# Regressão 

reg_barium = smf.ols('np.log(chnimp) ~ np.log(chempi) + np.log(gas) + np.log(rtwex) + befile6 + affile6 + afdec6',
                     data = barium)
results_reg_barium = reg_barium.fit()
print(results_reg_barium.summary())

                            OLS Regression Results                            
Dep. Variable:         np.log(chnimp)   R-squared:                       0.305
Model:                            OLS   Adj. R-squared:                  0.271
Method:                 Least Squares   F-statistic:                     9.064
Date:                Tue, 06 Sep 2022   Prob (F-statistic):           3.25e-08
Time:                        15:30:44   Log-Likelihood:                -114.79
No. Observations:                 131   AIC:                             243.6
Df Residuals:                     124   BIC:                             263.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -17.8028     21.045     -0.

**Teste do multiplicador de Lagrange (Breusch-Godfrey test)**: 

- Statsmodels: https://www.statsmodels.org/dev/generated/statsmodels.stats.diagnostic.acorr_breusch_godfrey.html

In [6]:
# Computando os testes de autocorrelação (automaticamente):

bg_test_result = acorr_breusch_godfrey(results_reg_barium, nlags = 3) # tupla com (lm, lmpval, fval, fpval)
fstat_auto = bg_test_result[2]
fpval_auto = bg_test_result[3]
print(f'F stat: {fstat_auto}\n')
print(f'pval: {fpval_auto}\n')

F stat: 5.124662239772489

pval: 0.0022637197671316546



In [7]:
# Teste com a regressão os resíduos:

# (I) Calculando os resíduos e criando da primeira a terceira defasagem:

barium['resid'] = results_reg_barium.resid
barium['resid_lag1'] = barium['resid'].shift(1)
barium['resid_lag2'] = barium['resid'].shift(2)
barium['resid_lag3'] = barium['resid'].shift(3)

# (II) Regressão do resíduo nas variáveis exógenas e nas suas três defasagens:

reg_barium_resid = smf.ols('resid ~ resid_lag1 + resid_lag2 + resid_lag3 +'
                           'np.log(chempi) + np.log(gas) + np.log(rtwex) +'
                           'befile6 + affile6 + afdec6', 
                           data = barium)
results_reg_barium_resid = reg_barium_resid.fit()

# (III) Testando as hipóteses:

hypothesis = ['resid_lag1 = 0', 'resid_lag2 = 0', 'resid_lag3 = 0']
ftest_manual = results_reg_barium_resid.f_test(hypothesis)
fstat_manual = ftest_manual.statistic
fpval_manual = ftest_manual.pvalue
print(f'fstat_manual: {fstat_manual}\n')
print(f'fpval_manual: {fpval_manual}\n')

fstat_manual: 5.122907054069386

fpval_manual: 0.002289802832966284



In [8]:
# Estimando por MQG com Cochrane-Orcutt (procedimento com interações):
# OBS: No livro, usa-se a estimação Prais-Winsten, mas a dinâmica de ambas é similar e assintóticamente as diferenças são irrelevantes

y, X = pt.dmatrices('np.log(chnimp) ~ np.log(chempi) + np.log(gas) +'
                    'np.log(rtwex) + befile6 + affile6 + afdec6',
                    data=barium, return_type='dataframe')
reg = sm.GLSAR(y, X)
CORC_results = reg.iterative_fit(maxiter = 100)
table = pd.DataFrame({'b_CORC': CORC_results.params,
                      'se_CORC': CORC_results.bse})
print(f'reg.rho: {reg.rho}\n')
print(f'table: \n{table}\n')

reg.rho: [0.29585313]

table: 
                   b_CORC    se_CORC
Intercept      -37.512978  23.239015
np.log(chempi)   2.945448   0.647696
np.log(gas)      1.063321   0.991558
np.log(rtwex)    1.138404   0.514910
befile6         -0.017314   0.321390
affile6         -0.033108   0.323806
afdec6          -0.577328   0.344075



> Vale lembrar que nomralmente, tanto para a estimação Prais-Winsten quanto para a Cochrane-Orcutt os erros-padrões são maiores do que na estimação por MQO, dado que nos dois primeiros casos, considera-se a autocorrelação, enquanto em MQO ela não é computada na estimação. Isso ocorre porque **os estimadores de MQO geralmente subestimam a variação amostral dos dados com autocorrelação, produzindo erros-padrões menores.** 

> Nota-se que os parâmetros estatisticamente significantes não deixam de assumir essa condição com a mudança de método de estimação, dada a sua relevância para explicar o fenômeno. 

> O coeficiente de determinação $R^2$ também é inferior nas estimações Prais-Winsten e Cochrane-Orcutt em relação à regressão estimada por MQO. Contudo, a comparação é equivocada, pois o coeficiente de determinação para MQO é calculado com os dados sem tranformações, enquanto nos dois métodos citados acima os dados são tranformados. Ainda, para eles, é difícil estabelecer uma interpetação clara, dadas as tranformações feitas nos dados, mas mesmo assim o $R^2$ é reportado. 

## Exemplo 12.5: Static Phillips Curve - OLS X FGLS


In [9]:
# MQO:

print(results_s.summary())

                            OLS Regression Results                            
Dep. Variable:               Q("inf")   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     2.616
Date:                Tue, 06 Sep 2022   Prob (F-statistic):              0.112
Time:                        15:30:45   Log-Likelihood:                -124.43
No. Observations:                  49   AIC:                             252.9
Df Residuals:                      47   BIC:                             256.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.4236      1.719      0.828      0.4

In [10]:
# MQGF:

phillips_96 = phillips[phillips['year'] <= 1996]
y, X = pt.dmatrices('Q("inf") ~ unem', data = phillips_96, return_type='dataframe')
reg = sm.GLSAR(y, X)
CORC_results = reg.iterative_fit(maxiter = 100)
table = pd.DataFrame({'b_CORC': CORC_results.params,
                      'se_CORC': CORC_results.bse})
print(f'reg.rho: {reg.rho}\n')
print(f'table: \n{table}\n')

reg.rho: [0.79072939]

table: 
             b_CORC   se_CORC
Intercept  7.696532  2.453932
unem      -0.689497  0.319084



> Podemos observar como os coeficientes são bruscamente diferentes: para a variável desemprego, muda-se inclusive o sinal, e não apenas o valor. Nesse caso, tendemos a ir com os coeficientes da estimação por MQGF (Cochrane-Orcutt), pois está de acordo com o sinal esperado pela teoria econômica, a Cruva de Phillips e o trade-off entre inflação e desemprego. 

> A estimação por MQGF é similar, em certo ponto, com o estimado por MQO com as diferenciações, obtendo a Curva de Phillips aceleracionista. Isso porque a estimação com MQGF utiliza de uma diferenciação parcial, que se aproxima da primeira diferenciação. Confira abaixo:

In [11]:
print(results_ea.summary())

                            OLS Regression Results                            
Dep. Variable:              inf_diff1   R-squared:                       0.108
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     5.558
Date:                Tue, 06 Sep 2022   Prob (F-statistic):             0.0227
Time:                        15:30:45   Log-Likelihood:                -110.12
No. Observations:                  48   AIC:                             224.2
Df Residuals:                      46   BIC:                             228.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.0306      1.377      2.201      0.0

> Para esse caso, podemos observar que para processos não-estacionário, que possuem raiz unitária, a estimação por MQGF pode ser mais adequada que por MQO. 

## Comparando MQO e MQGF

Em termos práticos, e não apenas teóricos, os estimadores de MQO e MQGF podem ser bem diferentes.

- Os estimadores de MQGF precisam que os dados em t, na sua primeira defasagem e na sua observação seguinte não sejam correlacionados com o termo do erro.

- Geralmente, os estimadores podem ser bem diferentes para os dois métodos justamente porque a hipótese acima falha.

- Nesse caso, temos duas alternativas:

    - Se os coeficientes estimados forem muito distintos, devemos optar pelos estimadores de MQO, pois mesmo com autocorrelação, se as variáveis exógenas não estiverem correlacionadas com o termo do erro, teremos que os estimadores ainda serão consistentes assintóticamente. O mesmo não ocorre para MQGF, pois necessita da hipótese adicional colocada acima, que geralmente falha.

    - Se os coeficientes estimados forem similares podemos assumir que a hipótese acima apresentada se mantém de alguma forma, e por isso os estimadores de MQGF são preferíveis por serem mais eficientes sob presença de forte evidência para a correlação serial.

- Ainda, podemos testar se a diferenças dos estimadores é realmente significante do ponto de vista estatístico, usando o teste de Hausmann, que vimos para estimadores com efeitos fixos e aleatórios para dados em painel.

## Corrigindo para correlação serial de ordens superiores

- Assumindo condições de estabilidade, podemos tranformar os dados para eliminar a correlação serial, como feito no caso mais simples.

- Tomamos os mesmo procedimentos:
    - Regressão com MQO e extração dos resíduos;

    - Regressão dos resíduos em suas defasagens e nas variáveis independentes;

    - Obtenção dos rhos e dos testes significância

    - Transformação dos dados e nova regressão com MQO.

- Esse procedimento é análogo a estimação de MQGF e eliminam a correlação serial para o número de observações escolhido (2 ou mais) e tornamos erros homoscedasticos.

# Vale lembrar

1. Processo integrado de ordem zero **I(0)** --> Processo/série estacionária ou com dependência fraca;

2. Processo integrado de ordem um **I(1)** --> Processo/série não-estacionária, mas que sua primeira diferenciação transforma-se em uma série com dependência fraca (ou estacionária);

# Diferenciação (decomposição) e Correlação Serial
---

Os processos de diferenciação (decomposição) também possuem méritos quando lidamos com séries persistentes, dada a sua capacidade de tornar processos integrados em processos de dependência fraca. 

- A inferência com os estimadores de MQO sem a diferenciação dos dados pode nos levar a conclusões erradas, principalmente se os processos são integrados em primeira ordem I(1) ou superiores. O passeio aleatório é um dos exemplos.

- Assim, a **diferenciação é uma ferramenta interessante para eliminar a correlação serial ou parte dela.**

## Exemplo 12.6: Differencing the Interest Rate Equation

In [12]:
intdef = woo.data('intdef')

In [13]:
# Regressão OLS

irate_regOLS = smf.ols(formula='i3 ~ Q("inf") + Q("def")', data = intdef)
results_irate_regOLS = irate_regOLS.fit()
print(results_irate_regOLS.summary())
intdef['resid'] = results_irate_regOLS.resid
intdef['resid_lag1'] = intdef['resid'].shift(1)

                            OLS Regression Results                            
Dep. Variable:                     i3   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     40.09
Date:                Tue, 06 Sep 2022   Prob (F-statistic):           2.48e-11
Time:                        15:30:46   Log-Likelihood:                -112.16
No. Observations:                  56   AIC:                             230.3
Df Residuals:                      53   BIC:                             236.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7333      0.432      4.012      0.0

In [14]:
# Regressão com os resíduos para verificar a correlação serial

reg_irate_resid = smf.ols('resid ~ resid_lag1', data = intdef)
results_reg_irate_resid = reg_irate_resid.fit()
print(results_reg_irate_resid.summary())

                            OLS Regression Results                            
Dep. Variable:                  resid   R-squared:                       0.377
Model:                            OLS   Adj. R-squared:                  0.366
Method:                 Least Squares   F-statistic:                     32.13
Date:                Tue, 06 Sep 2022   Prob (F-statistic):           6.06e-07
Time:                        15:30:46   Log-Likelihood:                -95.938
No. Observations:                  55   AIC:                             195.9
Df Residuals:                      53   BIC:                             199.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0153      0.190      0.081      0.9

O coeficiente $\rho_1$ para a primeira defasagem do resíduo indica a presença de forte correlação serial, o que nos leva a necessidade de corrigir o problema. Vamos utilizar da diferenciação para contornar a autocorrelação.

In [15]:
intdef['i3_diff1'] = intdef['i3'].diff()
intdef['inf_diff1'] = intdef['inf'].diff()
intdef['def_diff1'] = intdef['def'].diff()

In [16]:
# Rodando novamente a regressão com os dados diferenciados

irate_regdiff = smf.ols(formula = 'i3_diff1 ~ inf_diff1 + def_diff1', data = intdef)
results_irate_regdiff = irate_regdiff.fit()
print(results_irate_regOLS.summary(), results_irate_regdiff.summary(), sep = '\n\n')

                            OLS Regression Results                            
Dep. Variable:                     i3   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.587
Method:                 Least Squares   F-statistic:                     40.09
Date:                Tue, 06 Sep 2022   Prob (F-statistic):           2.48e-11
Time:                        15:30:46   Log-Likelihood:                -112.16
No. Observations:                  56   AIC:                             230.3
Df Residuals:                      53   BIC:                             236.4
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.7333      0.432      4.012      0.0

> Podemos visualizar que os coeficientes são bem diferentes entre as regressões com os dados originais em nível e os dados diferenciados. Isso é uma evidência de que as variáveis exógenas não são estritamente exógenas, trazendo o problema da endogeneidade, ou que as variáveis apresentam raiz unitária. 

In [17]:
# Conferindo a correlação serial da nova regressão

intdef['resid_diff1'] = results_irate_regdiff.resid
intdef['resid_diff1_lag1'] = intdef['resid_diff1'].shift(1)

irate_regdiff_resid = smf.ols(formula = 'resid_diff1 ~ resid_diff1_lag1', data = intdef)
results_irate_regdiff_resid = irate_regdiff_resid.fit()
print(results_irate_regdiff_resid.summary())

                            OLS Regression Results                            
Dep. Variable:            resid_diff1   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.2871
Date:                Tue, 06 Sep 2022   Prob (F-statistic):              0.594
Time:                        15:30:46   Log-Likelihood:                -86.478
No. Observations:                  54   AIC:                             177.0
Df Residuals:                      52   BIC:                             180.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.0414      0.166  

> Concluímos que eliminamos a correlação serial dos erros dado o pequeno coeficiente $\rho_1$ da lag do resíduo e graças ao resultado do teste t, que nos dá mais e 50% de chance de erro ao rejeitarmos a hipótese nula de que não há autocorrelação. 

> Ou seja, **a diferenciação nos auxiliou a retiar a raiz unitária das séries analisadas, tornado-as estacionárias, e nos ajudou a controlar a correlação serial.**

# Correlação serial e Inferência Robsuta após MQO
---

- Por alguns dos impasses citados com o MQGF, tornou-se mais popular estimar os modelos por MQO mesmo com correlação serial, corringindo esse problema e o da heteroscedasticidade com o uso dos erros-padrões robustos dos estimadores.

Os **estimadores de MQO serão ineficientes**, mas algumas das explicações para essa abordagem são:

1. Na maioria das aplicações, as variáveis não são estritamente exógenas, o que deixa MQGF inferior ao MQO, pois nesse caso, não serão nem eficientes e nem consistentes;

2. Em MQGF os erros são assumidos como um processo autorregressivo de ordem 1 AR(1). Em MQO podemos generalizar com os erros-padrões robustos para outras formas.

3. No caso de MQO, para calcularmos a varância dos erros, basta plugar $\rho$ na relação derivada no livro texto, mas aqui assumimos que há homoscedasticidade. Mas essa hipótese pode ser relaxada.

## Lidando com erros heteroscedasticos e autoccorelacionados

Abordagem robusta desenvolvida por **Davidson e MacKinnon (1993)**.

1. Estimamos a regressão da maneira convencional;

2. Prosseguimos com uma regressão de uma das variáveis exógenas contra as demais exógenas (podemos repetir o processo para mais de uma, se necessário);

3. Então, podemos mostrar que a variância assintótica robusta do estimador depende do erro dos passos 1 e 2 acima. 

$$
AVar(\beta_1) = (\sum_{t = 1}^{n} E(r_{t}^2))^{-2} Var(\sum_{t = 1}^{n} r_t u_t)
$$

**Com ou sem correlação serial, chegaremos ao erro padrão robusto que serve para ambos os casos.** Então, consideramos que quando a correlação não é persistente para muitos períodos a frente, conseguimos fazer a inferência robusta controlando a correlação para períodos curtos com essa relação. Então, a dependência fraca é interessante para esse tipo de problema, dado que espera-se que nessas condições a correlação serial reduza rapidamente para períodos curtos. 

Com isso, Newey e West (1987) e Wooldridge (1989) indicam uma forma de estimar a $AVar$ e o erro-padrão dos estimadores a partir da relação acima. 
- **[VERIFICAR DERIVAÇÃO COMPLETA NO LIVRO]**

A partir desses procedimentos, temos erros-padrões robustos com viabilidade para a relaização dos testes $t$ e $F$. Nesse caso, a literatura de séries temporais os chama de *Erro-padrão consistente à autocorrelação e heteroscedasticidade*, em inglês: ***heteroskedasticity and autocorrelation consistent standard errors (HAC standard errors)***.

Podemos ajustar essa relação para diferentes tipos de correlação serial e diferentes tamanhos de amostra. Além disso, dependendo do tamanho da amostra podemos ser mais flexíveis em relação à correlação serial. Newey e West (1987) recomendam a inclusão do número de termos para a correlação serial $g$ como:

$$
4(n/100)^{2/9}
$$

Outra "regra" seria:

$$
n^{1/4}
$$

## Procedimento para computar o erro padrão para um estimador qualquer

1. Estimar a regressão por MQO normalmente;

2. Computar os resíduos da regressão auxiliar (regressão da variável independente correspondente ao estimador analisado nas demais variáveis independentes) e incorporar um termo de erro composto $\hat{a_t} = \hat{r_t}\hat{u_t}$ para cada t, sendo r e u os erros da regressão auxiliar e da regressão inicial, respectivamente.

3. Escolher $g$ por uma das regras listadas acima e computar $\hat{v}$, o erro padrão robusto à correlação serial;

4. Computar o erro padrão do estimador pela regra dada no livro texto, equação [12.43]

### Diferenças entre o erro padrão e o erro padrão robusto e impasses

Além das diferenças teóricas já citadas, pelo erro padrão vir de um processo de estimação que ignora a correlação serial, espera-se, na maioria dos casos, que empiricamente o erro padrão robusto seja maior.  

Vale lembrar que quando temos correlação serial substancial somada a um tamanho pequeno de amostra, os erros padrões robustos podem performar mal. Dependendo da intensidade da correlação serial, temos que os coeficientes podem ser menos significativos do que pensamos, do ponto de vista estatístico, ou mesmo não significantes. 

Se estivermos confiantes em relação a exogeneidade estrita das variáveis independentes, por mais que seja uma hipótese forte, e sejamos céticos em relação aos erros seguirem um processo AR(1), podemos estimar por MQGF com Prais-Winsten ou Cochrane-Orcutt. Isso porque com o processo de decomposição parcial desses métodos ainda podemos obter estimadores do que em MQO, sob as condições descritas acima. Mas se os erros de fato não seguirem um processo AR(1), então os erros padrões estimados estarão incorretos. 

Em outros casos, podemos ainda decompor parcialmente a série de forma manual, estimar por MQO e então calcular os erros padrões robustos. Esse procedimento elimina boa parte da correlação serial e é semelhante ao que fazemos com Mínimos Quadrados Ponderados na presença de heteroscedasticidade.

> ***DOCUMENTAÇÃO E EXEMPLOS DE COMO IMPLEMENTAR ERRO-PADRÃO ROBUSTO:*** https://stackoverflow.com/questions/30553838/getting-statsmodels-to-use-heteroskedasticity-corrected-standard-errors-in-coeff

# Heteroscedasticidade em Séries Temporais
---

Assim como como em dados cross-section, a heteroscedasticidade não provoca viés ou incosistência nos estimadores, mas **torna inválidos os erros padrões, as estatísticas t e F**, inviabilizando a inferência. 

Geralmente, temos que os problemas de correlação serial são mais relevantes do que os problemas de heteroscedasticidade em séries temporais, isso porque algumas ferramentas que usamos para corrigir a correlação serial podem corrigir a heteroscedasticidade, mas o contrário não ocorre. Ainda, os problemas vindos da correlação serial são mais relevantes. 

## Correções

Em relação as correções, podemos seguir com os seguintes procedimentos:

1. Para corrigir as estatísticas de teste, basta seguirmos o mesmo procedimento do que fizemos com dados cross-section, utilizando os erros padrões e as estatísticas de teste robustas à heteroscedasticidade. Os pacotes econométricos fornecem com facilidade.

As vezes, queremos testar para a heteroscedasticidade em aplicações expecíficas, principalmente quando estamos lidando com pequenas amostras. Mas vale lembrar que para testarmos para heteroscedasticidade, a **série não poder conter correlação serial, caso contrário, o teste será invalidado**. Dessa forma, faz sentido **testar para correlação serial antes de testar para heteroscedasticidade**.

Dessa forma:

2. Podemos conduzir o teste de Breusch-Pagan, com a regressão dos resíduos ao quadrado nas variáveis independentes da regressão inicial. Vale lembrar que o erro dessa regressão deve ser homoscedastico e não correlacionado serialmente. 

Esse procedimento deve ser considerado padrão para outros testes também, como o teste de White. Ainda, se houver heteroscedasticidade no erro e não houver correlação serial, podemos prosseguir com as estatísticas de teste robustas à heteroscedasticidade.  

3. Outra alternativa é conduzir uma regressão com estimadores de mínimos quadrados ponderados (MQP ou WLS em inglês) e comparar os estimadores e as estatísticas de teste com MQO. Esse procedimento é comum ao que já realizamos para dados cross section. 

## Example 12.8: Heteroskedasticity and the Efficient Markets Hypothesis

In [18]:
nyse = woo.data('nyse')
nyse['ret'] = nyse['return']
nyse['ret_lag1'] = nyse['return_1']
nyse

,price,return,return_1,t,price_1,price_2,cprice,cprice_1,ret,ret_lag1
0,49.750000,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN
1,51.439999,3.396982,NaN,2,49.750000,NaN,1.689999,NaN,3.396982,NaN
2,52.049999,1.185849,3.396982,3,51.439999,49.750000,0.610001,1.689999,1.185849,3.396982
3,52.279999,0.441882,1.185849,4,52.049999,51.439999,0.230000,0.610001,0.441882,1.185849
4,54.240002,3.749049,0.441882,5,52.279999,52.049999,1.960003,0.230000,3.749049,0.441882
...,...,...,...,...,...,...,...,...,...,...
686,161.740005,-1.106694,-1.004784,687,163.550003,165.210007,-1.809998,-1.660004,-1.106694,-1.004784
687,165.449997,2.293799,-1.106694,688,161.740005,163.550003,3.709991,-1.809998,2.293799,-1.106694
688,166.669998,0.737384,2.293799,689,165.449997,161.740005,1.220001,3.709991,0.737384,2.293799
689,163.410004,-1.955958,0.737384,690,166.669998,165.449997,-3.259995,1.220001,-1.955958,0.737384


In [19]:
# Regressão principal

heteros_efh_reg = smf.ols('ret ~ ret_lag1', data = nyse)
results_heteros_efh_reg = heteros_efh_reg.fit()

# Obtendo os resíduos ao quadrado

nyse['resid'] = results_heteros_efh_reg.resid
nyse['resid_sq'] = nyse['resid'] ** 2

# Regressão para teste de heteroscedasticidade

u_sq_reg = smf.ols('resid_sq ~ ret_lag1', data = nyse)
results_u_sq_reg = u_sq_reg.fit()
print(results_u_sq_reg.summary())

                            OLS Regression Results                            
Dep. Variable:               resid_sq   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     30.05
Date:                Tue, 06 Sep 2022   Prob (F-statistic):           5.90e-08
Time:                        15:30:47   Log-Likelihood:                -2639.9
No. Observations:                 689   AIC:                             5284.
Df Residuals:                     687   BIC:                             5293.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.6565      0.428     10.888      0.0

# Heteroscedasticidade condicional autorregressiva (Autoregressive Conditional Heteroskedasticity, ARCH)
---

A heteroscedasticidade é um dos pontos de atenção nas regressões e podemos verificá-la em sua forma dinâmica até em modelos sem dinâmica na equação da regressão.

O modelo ARCH de primeira ordem é dado por:

$$
E(u_t^2|u_{t-1}, u_{t-1}, ...) = E(u_t^2|u_{t-1}) = \alpha_0 + \alpha_1u_{t-1}^2 + v_t
$$

se $\alpha_1 = 0$, não há dinâmica e o modelo não faz sentido. Logo, $\alpha_1 \geq 0$ e $\alpha_0 > 0$ 

Ainda, os termos do erro ao quadrado na regressão possuem correlação serial, mesmo que nasua forma linear não haja correlação serial.

- Condição de estabilidade $\alpha_1 < 0$

## Quais as implicações para os estimadores de MQO?

Se as hipóteses Gauss-Markov se mantiverem, teremos que os estimadores de MQO ainda serão consistentes e suas estatísticas de testes serão válidas.

Contudo, mesmo com as propriedades desejadas, ainda devemos nos preocupar com duas situações no caso de estimadores de MQO:

1. Podemos obter estimadores assintóticamente mais eficientes e consistentes (embora não necessariamente não-viesados): como por MQP ou máxima verossimilhança se os erros se distribuírem normalmente.

2. Em alguns campos da economia a variância condicional, associada ao erro, é relevante. Como a variância é utilizada para medir volatilidade, os moelos ARCH ganharam destaque principalmente na área das finanças. 

Ainda, vale lembrar que o modelo ARCH também se aplica para a dinâmica na média condicional. Mas novamente, sob as hipótese Gauss-Markov MQO ainda terá as propriedades desejadas. 



## Example 12.9: ARCH in Stock Returns

In [20]:
# Criando a primeira defasagem do erro ao quadrado

nyse['resid_sq_lag1'] = nyse['resid_sq'].shift(1)

# ARCH model nos retornos das ações

arch_reg = smf.ols('resid_sq ~ resid_sq_lag1', data = nyse)
results_arch_reg = arch_reg.fit()
print(results_arch_reg.summary())

                            OLS Regression Results                            
Dep. Variable:               resid_sq   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.112
Method:                 Least Squares   F-statistic:                     87.92
Date:                Tue, 06 Sep 2022   Prob (F-statistic):           9.71e-20
Time:                        16:04:07   Log-Likelihood:                -2609.7
No. Observations:                 688   AIC:                             5223.
Df Residuals:                     686   BIC:                             5233.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         2.9474      0.440      6.695

> Com o coeficiente da primeira defasagem do resíduo ao quadrado e sua estatística de teste, podemos ver que há forte evidência para a heteroscedasticidade dinâmica e a volatilidade de hoje depende da volatilidade passada. Os resíduos ao quadrado da regressão possuem correlação serial, mas os resíduos em sua forma comum não, o que também é uma evidênica, até certo ponto, da validade da hipóstese dos mercados eficientes ao longo do tempo. O retorno de hoje não está correlacionado com o passad, mas a variância nos retornos de hoje está correlacionada com a variância no passado. 

# Heteroscedasticidade e Correlação serial em modelos de regressão
---

Vale destacar também a possibilidade de ocorrência de heteroscedasticidade e correlação serial, juntas. Sem a certeza absoluta, podemos sempre prosseguir com MQO computandos os erros padrões robustos. 

Vale ressaltar que, geralmente, a correlação serial é considerada como o maior problema nesses casos, pois seu impacto sobre a consistência e a validade das estatisticas de testes são mais relevantes.

Para esse problema mais relevante, podemos prosseguir com os testes de identificação robustos e, se for o caso, utilizar as estimações por e Cochrane-Orcutt ou Prais-Winsten com a decomposição parcial da série, ou podemos realizar uma decomposição e estimar por MQO usando as estatísticas robustas de teste. Ou podemos realizar os testes de Breusch-Pagan ou White. 

Podemos ainda corrigir ambos os problemas de uma só vez, com a estimação por Mínimos Quadrados Ponderados junto da abordagem AR(1). 

## MQGF e AR(1) com correlação serial

Na maioria dos casos, devemos prosseguir com os seguintes passos:

1. Estimar por MQO e armazenar os resíduos;

2. Regredir $log(hat{u_t^2}$ nas variáveis exógenas ou na endógena normal e ao quadrado e obter os valores ajustados g;

3. Obter as estimativas de h com a forma $hat{h_t} = exp(g_t)$

4. Estimar a equação transformada com CO ou PW. 

Assim, teremos estimadores eficientes e as estatísticas de teste serão válidas.